In [1]:
comp_name = "amex-default-prediction"
nb_name = 'kagglenb004ti-amex-lgbm-dart'

import sys
import os
from pathlib import Path

if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount("/content/drive")
    base = f"/content/drive/MyDrive/colab_notebooks/kaggle/{comp_name}/notebooks"
    %cd {base}

KAGGLE_ENV = True if 'KAGGLE_URL_BASE' in set(os.environ.keys()) else False
INPUT_DIR = Path('../input/')

if KAGGLE_ENV:
    OUTPUT_DIR = Path('')
else:
    !mkdir ../input/{nb_name}
    OUTPUT_DIR = INPUT_DIR / nb_name

Mounted at /content/drive
/content/drive/MyDrive/colab_notebooks/kaggle/amex-default-prediction/notebooks
mkdir: cannot create directory ‘../input/kagglenb004ti-amex-lgbm-dart’: File exists


In [2]:
# THRES = 0.999999
THRES = 0.9

# Comments:
    
This is an improvement of my baseline, you can find it here: https://www.kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7963

The main difference between this solution and previous one is that we add new features and do seed blend to boost LB. Single 5 kfold model using seed 42 achieve an out of folds CV of 0.7977 and a public leaderboard of 0.799. If we use seed blend (train three different models using seed 42, 52, 62 and then average predictions) the LB boost niceley.

The main features that boost CV are the following:

* The difference between last value and the lag1
* The difference between last value and the average (this features gives a nice boost)

This feature engineer is done on all the last columns, so we actually add a lot of features, this model used 1368 features.

I uploaded test predictions to avoid running training and inference

Next Steps:

* Could try feature selection, maybe a lot of the feature are just noise, actually I perform permutation importance and I reduce the amount of features to 1000 app and the CV was almost the same. Maybe there is a better feature selection technique that can boost performance.

* Could try different models, maybe some neural network with the same features or a subset of the features and then blend with LGBM can work, in my experience blending tree models and neural network works great because they are very diverse so the boost is nice

* Could try more feature engineering, maybe we can create more features that extract the hidden signal of the dataset, actually I would first work on this option and really try to capture all the signal that the dataset has.

# Preprocessing

In [3]:
# ====================================================
# Library
# ====================================================
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

# ====================================================
# Get the difference
# ====================================================
def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1

# ====================================================
# Read & preprocess data and save it to disk
# ====================================================
def read_preprocess_data():
    train = pd.read_parquet(INPUT_DIR / 'amex-data-integer-dtypes-parquet-format/train.parquet')
    
    #### RIOW
    print('train.shape before filtering: ', train.shape)
    train_adv = pd.read_parquet(INPUT_DIR / 'kagglenb003-adversarial-validation/train_adv_valid.parquet')
    train_adv = train_adv[['customer_ID', 'private']]
    train = pd.merge(train, train_adv, how='inner', on='customer_ID')
    del train_adv
    gc.collect()
    train = train[train['private'] >= THRES]
    train.drop('private', axis=1, inplace=True)
    print('train.shape after filtering: ', train.shape)
    #### RIOWRIOW

    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_features = [col for col in features if col not in cat_features]
    print('Starting training feature engineer...')
    train_num_agg = train.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)
    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    train_labels = pd.read_csv(INPUT_DIR / 'amex-default-prediction/train_labels.csv')
    # Transform float64 columns to float32
    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        train_num_agg[col] = train_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        train_cat_agg[col] = train_cat_agg[col].astype(np.int32)
    # Get the difference
    train_diff = get_difference(train, num_features)
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')
    del train_num_agg, train_cat_agg, train_diff
    gc.collect()


    test = pd.read_parquet(INPUT_DIR / 'amex-data-integer-dtypes-parquet-format/test.parquet')
    print('Starting test feature engineer...')
    test_num_agg = test.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    test_num_agg.reset_index(inplace = True)
    test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    test_cat_agg.reset_index(inplace = True)
    # Transform float64 columns to float32
    cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        test_num_agg[col] = test_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        test_cat_agg[col] = test_cat_agg[col].astype(np.int32)
    # Get the difference
    test_diff = get_difference(test, num_features)
    test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID').merge(test_diff, how = 'inner', on = 'customer_ID')
    del test_num_agg, test_cat_agg, test_diff
    gc.collect()
    # Save files to disk
    train.to_parquet(OUTPUT_DIR / f'train_fe_adv{THRES}.parquet')
    test.to_parquet(OUTPUT_DIR / 'test_fe.parquet')


In [ ]:
# Read & Preprocess Data

# train.shape before filtering:  (5531451, 190)
# train.shape after filtering:  (5218818, 191)

# read_preprocess_data()

train.shape before filtering:  (5531451, 190)
train.shape after filtering:  (2439599, 190)
Starting training feature engineer...


  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/192978 [00:00<?, ?it/s]

Starting test feature engineer...


  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/924621 [00:00<?, ?it/s]

# Training & Inference

In [ ]:
!pip uninstall --yes lightgbm && pip install --install-option=--gpu lightgbm

Found existing installation: lightgbm 2.2.3
Uninstalling lightgbm-2.2.3:
  Successfully uninstalled lightgbm-2.2.3
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 14.2 MB/s 
Skipping wheel build for lightgbm, due to binaries being disabled for it.
    Running setup.py install for lightgbm ... done


In [4]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

# ====================================================
# Configurations
# ====================================================
class CFG:
    #input_dir = '/content/data/'
    seed = 42
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    CFG.seed = seed

# ====================================================
# Read data
# ====================================================
def read_data():
    train = pd.read_parquet(OUTPUT_DIR / f'train_fe_adv{THRES}.parquet')
    train_full = pd.read_parquet(OUTPUT_DIR / f'train_fe.parquet')
    test = pd.read_parquet(OUTPUT_DIR / 'test_fe.parquet')
    return train, train_full, test

# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

# ====================================================
# LGBM amex metric
# ====================================================
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

# ====================================================
# Train & Evaluate
# ====================================================
def train_and_evaluate(train, train_full, test):
    # Label encode categorical features
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    cat_features = [f"{cf}_last" for cf in cat_features]
    for cat_col in cat_features:
        encoder = LabelEncoder()
        #### RIOW
        encoder = encoder.fit(train_full[cat_col])
        # train[cat_col] = encoder.fit_transform(train[cat_col])
        train[cat_col] = encoder.transform(train[cat_col])
        #### RIOWRIOW
        test[cat_col] = encoder.transform(test[cat_col])
    # Round last float features to 2 decimal place
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
        test[col + '_round2'] = test[col].round(2)
    # Get the difference between last and mean
    num_cols = [col for col in train.columns if 'last' in col]
    num_cols = [col[:-5] for col in num_cols if 'round' not in col]
    for col in num_cols:
        try:
            train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
            test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
        except:
            pass
    # Transform float64 and float32 to float16
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    for col in tqdm(num_cols):
        train[col] = train[col].astype(np.float16)
        test[col] = test[col].astype(np.float16)
    # Get feature list
    features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
    params = {
        'objective': 'binary',
        'metric': CFG.metric,
        'boosting': CFG.boosting_type,
        'seed': CFG.seed,
        'num_leaves': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.20,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 2,
        'min_data_in_leaf': 40,
        #'device': 'gpu',
        #'gpu_platform_id': 0,
        #'gpu_device_id': 0
        }
    # Create a numpy array to store test predictions
    test_predictions = np.zeros(len(test))
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train))
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
        print(' ')
        print('-'*50)
        print(f'Training fold {fold} with {len(features)} features...')
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
        lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
        lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
        model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 10500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 1500,
            verbose_eval = 500,
            feval = lgb_amex_metric
            )
        # Save best model
        joblib.dump(model, OUTPUT_DIR / f'lgbm_{CFG.boosting_type}_fold{fold}_seed{CFG.seed}_adv{THRES}.pkl')
        # Predict validation
        val_pred = model.predict(x_val)
        # Add to out of folds array
        oof_predictions[val_ind] = val_pred
        # Predict the test set
        test_pred = model.predict(test[features])
        test_predictions += test_pred / CFG.n_folds
        # Compute fold metric
        score = amex_metric(y_val, val_pred)
        print(f'Our fold {fold} CV score is {score}')
        del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
        gc.collect()
    # Compute out of folds metric
    score = amex_metric(train[CFG.target], oof_predictions)
    print(f'Our out of folds CV score is {score}')
    # Create a dataframe to store out of folds predictions
    oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
    oof_df.to_csv(OUTPUT_DIR / f'oof_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}_adv{THRES}.csv', index = False)
    # Create a dataframe to store test prediction
    test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
    test_df.to_csv(OUTPUT_DIR / f'test_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}_adv{THRES}.csv', index = False)

In [ ]:
# import importlib
# importlib.reload(lgb)

# import lightgbm as lgb

In [ ]:
train, train_full, test = read_data()

# seeds = [42, 1192, 2022]
# for seed in seeds:
#     seed_everything(seed)
#     train_and_evaluate(train, test)

seed_everything(1192)
train_and_evaluate(train, train_full, test)

  0%|          | 0/1080 [00:00<?, ?it/s]

 
--------------------------------------------------
Training fold 0 with 1365 features...
[500]	training's binary_logloss: 0.313855	training's amex_metric: 0.851562	valid_1's binary_logloss: 0.343498	valid_1's amex_metric: 0.75211
[1000]	training's binary_logloss: 0.216169	training's amex_metric: 0.904442	valid_1's binary_logloss: 0.274469	valid_1's amex_metric: 0.768011
[1500]	training's binary_logloss: 0.160367	training's amex_metric: 0.94982	valid_1's binary_logloss: 0.251536	valid_1's amex_metric: 0.776121
[2000]	training's binary_logloss: 0.127778	training's amex_metric: 0.978056	valid_1's binary_logloss: 0.24515	valid_1's amex_metric: 0.778282
[2500]	training's binary_logloss: 0.101351	training's amex_metric: 0.993146	valid_1's binary_logloss: 0.242206	valid_1's amex_metric: 0.780983
[3000]	training's binary_logloss: 0.0832334	training's amex_metric: 0.998178	valid_1's binary_logloss: 0.241509	valid_1's amex_metric: 0.784049
[3500]	training's binary_logloss: 0.066717	training's 

# Blend

In [ ]:
seeds = [42, 1192, 2022]

pred = 0.0
for i,seed in enumerate(seeds):
    if i==0:
        blend = pd.read_csv(OUTPUT_DIR / f'test_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{seed}_adv{THRES}.csv')
        #blend['prediction'] /= len(seeds)
        pred = blend['prediction'].values / len(seeds)
    else:
        tmp = pd.read_csv(OUTPUT_DIR / f'test_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{seed}_adv{THRES}.csv', usecols=['prediction'])
        pred += tmp['prediction'].values / len(seeds)
        #pred /= len(seeds)
        #blend['prediction'] = blend['prediction'] + pred
        del tmp
        gc.collect()
blend['prediction'] = pred


blend.to_csv(OUTPUT_DIR / f'test_lgbm_baseline_5fold_seed_blend_adv{THRES}.csv', index=False)

In [ ]:
# Submit blend.csv to Kaggle dataset

# Read Submission File
This is the submission file corresponding to the output of the previous pipeline (using the average blend of 3 seeds)

In [ ]:
# sub = pd.read_csv('../input/amex-sub/test_lgbm_baseline_5fold_seed_blend.csv')
# sub.to_csv('test_lgbm_baseline_5fold_seed_blend.csv', index = False)